### Download file GGUF

In [55]:
import os
os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_key
os.environ["SERPER_API_KEY"] = "caabb8481fd9568ce9e4f6149139098b7fd91f85"
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document as lcDocument
from langchain_pinecone import PineconeVectorStore
from langchain_community.utilities import GoogleSerperAPIWrapper
from bs4 import BeautifulSoup
from readability import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests


In [8]:
import requests
import os

url = 'https://huggingface.co/mradermacher/Llama-3.1-MedPalm2-imitate-8B-Instruct-GGUF/resolve/main/Llama-3.1-MedPalm2-imitate-8B-Instruct.Q8_0.gguf?download=true'
save_directory = 'models'
filename = 'Llama-3.1-MedPalm2-imitate-8B-Instruct.Q8_0.gguf'

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

file_path = os.path.join(save_directory, filename)

if os.path.exists(file_path):
    print(f'Tệp {filename} đã tồn tại trong thư mục {save_directory}.')
else:
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))  # Total size in bytes
        block_size = 8192  # Download in chunks of 8KB
        progress = 0  # Track the progress

        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    file.write(chunk)
                    progress += len(chunk)
                    percent_complete = (progress / total_size) * 100
                    print(f'Tải xuống: {percent_complete:.2f}%', end='\r')

        print(f'\nTệp đã được tải xuống và lưu tại {file_path}')
    else:
        print('Có lỗi xảy ra trong quá trình tải xuống.')


Tải xuống: 100.00%
Tệp đã được tải xuống và lưu tại models\Llama-3.1-MedPalm2-imitate-8B-Instruct.Q8_0.gguf


### Call model via API (llama_cpp server) with Langchain

In [56]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(base_url="http://zep.hcmute.fit/7500/v1", api_key="llama.cpp")

### Define the Offline_RAG Class

In [77]:
class WebSearchDB():
    def __init__(self):
        self.embedding = HuggingFaceEmbeddings()
        self.db = self.build_db()
        self.text_slit = RecursiveCharacterTextSplitter(
                            separators = ["\n\n", "\n", " ", ".", ",", "\u200b", "\u3001", "\uff0e", "\u3002", ""],
                            chunk_size = 500,
                            chunk_overlap = 50,
                            length_function = len,
                            is_separator_regex = False)

    def build_db(self):
        db = PineconeVectorStore.from_existing_index(
            index_name="docs-rag-chatbot",
            namespace="docs-link",
            embedding=self.embedding
        )
        return db
    
    def is_not_existed_url(self, url):
        results = self.db.similarity_search(query=url, namespace="docs-link", k=1)
        if results[0].page_content == url:
            return False
        return True
    
    def store_pinecone(self, docs, namespace):
        if docs:
            db = PineconeVectorStore.from_documents(documents=docs, embedding=self.embedding, index_name="docs-rag-chatbot", namespace=namespace)

    def store_web_content(self, web_links):
        documents = []
        if web_links:
            for link in web_links:
                content, title = self.read_content_from_web(link)
                chunks = self.text_slit.split_documents(content)
                cleaned_chunks = self.clean_data(chunks)
            
                for i, chunk in enumerate(cleaned_chunks):
                    doc = lcDocument(
                        page_content=chunk,
                        metadata={"source": title, "chunk_id": i}
                    )
                    documents.append(doc)

            self.store_pinecone(documents, 'docs-web-content')

    def clean_data(self, chunks):
        cleaned_chunks = []
        for chunk in chunks:
            cleaned_content = chunk.page_content.replace("\n", " ")
            cleaned_content = re.sub(r'\s+', ' ', chunk.page_content)
            cleaned_content = cleaned_content.strip()
            cleaned_chunks.append(lcDocument(page_content=cleaned_content, metadata=chunk.metadata))
        return cleaned_chunks
        
    def read_content_from_web(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            doc = Document(response.content)
            html_content = doc.summary()
            title = doc.title()
            soup = BeautifulSoup(html_content, "html.parser")
            text_content = soup.get_text()
            return text_content, title
        return None, None

    def add_url(self, results):    
        links_document = []
        links = []
        for web in results['organic']:
            if self.is_not_existed_url(web['link']):
                link_doc = lcDocument(page_content=web['link'], metadata={'source': web['title']})
                links_document.append(link_doc)
                links.append(web['link'])
        self.store_pinecone(links_document, namespace='docs-link')
        self.store_web_content(links)

    def get_retriever(self,
                      search_type: str = 'similarity', 
                      search_kwargs: dict = {'k': 5}):
        retriever = self.db.as_retriever(search_type=search_type, search_kwargs=search_kwargs)
        return retriever

In [78]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain

class Offline_RAG:
    def __init__(self, llm) -> None:
        self.llm = llm

        self.history_chat =[]
        
        condense_question_template = """
        Với đoạn hội thoại sau và một câu hỏi tiếp theo, hãy diễn đạt lại câu hỏi tiếp theo để nó trở thành một câu hỏi độc lập.
        
        Lịch sử hội thoại:
        {chat_history}
        Câu hỏi tiếp theo: {question}
        Câu hỏi độc lập:"""
        
        self.condense_question_prompt = ChatPromptTemplate.from_template(condense_question_template)

        qa_template = """
        Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. 
        Dựa vào nội dung gợi ý trả lời bên dưới, hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi của họ.
        Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời.

        Lịch sử hội thoại:
        {chat_history}

        Nội dung gợi ý trả lời:
        {context}

        Câu hỏi: {question}
        """

        self.qa_prompt = ChatPromptTemplate.from_template(qa_template)

    def get_chain(self, retriever):
        rag_chain = ConversationalRetrievalChain.from_llm(
            llm,
            retriever,
            condense_question_prompt=self.condense_question_prompt,
            combine_docs_chain_kwargs={
                "prompt": self.qa_prompt,
            },
            return_source_documents=True,
        )
        return rag_chain
    
    def add_history(self, user_chat, bot_chat):
        self.history_chat.append(("user", user_chat))
        self.history_chat.append(("assistant", bot_chat))
        if len(self.history_chat) > 5:
            self.history_chat.pop(0)
    
    def get_input_data(self,question):
        search = GoogleSerperAPIWrapper()
        search.k = 2
        results = search.results(question)
        WebSearchDB().add_url(results=results)
        return {
            "chat_history": self.history_chat,
            "question": question
        }

### Define the VectorDB class using Pinecone

In [38]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

class VectorDB:
    def __init__(self,
                 embedding = HuggingFaceEmbeddings()) -> None:
        os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
        self.pinecone_api_key = os.getenv('PINECONE_API_KEY')
        self.embedding = embedding
        self.db = self._build_db()
    
    def _build_db(self):
        db = PineconeVectorStore.from_existing_index(
            index_name="docs-rag-chatbot",
            namespace="docs-store",
            embedding=self.embedding
        )
        return db
    
    # tính similarity
    def get_retriever(self,
                      search_type: str = "similarity",
                      search_kwargs: dict = {"k": 5}
                      ):
        retriever =self.db.as_retriever(search_type=search_type,
                                        search_kwargs=search_kwargs)
        return retriever

c:\1THUMUCDAIHOC\HOC_KY_7\PROJECT_AI\postpartum_psychology_sup_bot\.env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Function build chain

In [79]:
def build_rag_chain(llm):
    retriever = WebSearchDB().get_retriever()
    off_rag = Offline_RAG(llm)
    rag_chain = off_rag.get_chain(retriever)
    return rag_chain, off_rag

### Run example

#### Not prompt and context

In [7]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain
import time

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory=memory
)
sum_time = 0
n = 0
while True:
    question = input("You: ")

    if question == "exit":
        break

    start = time.time()
    response = conversation.invoke({
            "input": question
        })
    end = time.time()

    sum_time += end - start
    n += 1

    print(response)
    
    print(f"Time: {end - start}")

print(f"Average time: {sum_time/n}")


{'input': 'Đau đầu, mệt mỏi, suy nghĩ tiêu cực sau sinh là bị bệnh gì?', 'history': '', 'response': 'Chào bạn! Tôi hiểu bạn đang lo lắng về những triệu chứng bạn đang trải qua sau khi sinh nở.  Đau đầu, mệt mỏi và suy nghĩ tiêu cực sau sinh là những triệu chứng rất phổ biến được gọi là **bệnh trầm cảm sau sinh** hoặc **baby blues**. \n\nTuy nhiên, tôi không phải là chuyên gia y tế nên tôi không thể chẩn đoán bệnh.  Điều quan trọng nhất là bạn cần đến gặp bác sĩ để được kiểm tra và tư vấn điều trị phù hợp. Bác sĩ có thể giúp bạn xác định nguyên nhân của những triệu chứng này và đưa ra lời khuyên tốt nhất cho trường hợp của bạn. \n\nHãy nhớ rằng, bạn không đơn độc và rất nhiều phụ nữ khác cũng trải qua những cảm xúc tương tự sau khi sinh con.  Đừng ngần ngại tìm kiếm sự trợ giúp từ người thân, bạn bè hoặc các chuyên gia y tế.\n\n\n'}
Time: 91.38202548027039
{'input': 'Hiện tại tôi không có thời gian đi bác sĩ, con tôi cứ khóc hoài, có cách nào để cho nó ngủ ngoan không ?', 'history': 'Hu

#### Not context

In [8]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain_core.prompts import ChatPromptTemplate

memory = ConversationBufferMemory()
qa_template = """
        Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. 
        Hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi của họ.
        Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời.

        Lịch sử hội thoại:
        {history}

        Câu hỏi: {input}
        """
qa_prompt = ChatPromptTemplate.from_template(qa_template)

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    prompt = qa_prompt
)

sum_time = 0
n = 0

while True:
    question = input("You: ")
    if question == "exit":
        break
    start = time.time()
    response = conversation.invoke({
            "input": question
        })
    end = time.time()
    sum_time += end - start
    n += 1
    print(response)
    print(f"Time: {end - start}")

print(f"Average time: {sum_time/n}")

{'input': 'Đau đầu, mệt mỏi, suy nghĩ tiêu cực sau sinh là bị bệnh gì?', 'history': '', 'response': 'Tôi hiểu bạn đang cảm thấy rất khó khăn.  Đau đầu, mệt mỏi và suy nghĩ tiêu cực sau sinh là những triệu chứng thường gặp của **thoi ương trầm cảm** (postpartum depression).\n\nTuy nhiên, tôi không phải là chuyên gia y tế và không thể chẩn đoán bệnh. \n\n**Điều quan trọng nhất là bạn nên liên hệ với bác sĩ hoặc chuyên gia sức khỏe tâm thần để được tư vấn và hỗ trợ phù hợp.** Họ có thể giúp bạn xác định nguyên nhân của những triệu chứng bạn đang gặp phải và đưa ra phương pháp điều trị tốt nhất cho trường hợp của bạn.\n\nHãy nhớ rằng bạn không đơn độc và có rất nhiều nguồn lực sẵn sàng hỗ trợ bạn vượt qua giai đoạn này. \n'}
Time: 85.21116423606873
{'input': 'Hiện tại tôi không có thời gian đi bác sĩ, con tôi cứ khóc hoài, có cách nào để cho nó ngủ ngoan không ?', 'history': 'Human: Đau đầu, mệt mỏi, suy nghĩ tiêu cực sau sinh là bị bệnh gì?\nAI: Tôi hiểu bạn đang cảm thấy rất khó khăn.  Đ

#### Context

In [80]:
import time
chain, off_rag = build_rag_chain(llm)
sum_time = 0
n = 0
while True:
    question = input("You: ")
    if question == "exit":
        break
    
    start = time.time()
    response = chain.invoke(off_rag.get_input_data(question))
    end = time.time()

    sum_time += end - start
    n += 1

    print(response)
    
    off_rag.add_history(question, response["answer"])

    print(f"Time: {end - start}")

print(f"Average time: {sum_time/n}")

c:\1THUMUCDAIHOC\HOC_KY_7\PROJECT_AI\postpartum_psychology_sup_bot\.env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\1THUMUCDAIHOC\HOC_KY_7\PROJECT_AI\postpartum_psychology_sup_bot\.env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'chat_history': [], 'question': 'Làm sao để giảm stress', 'answer': 'Sau sinh, việc chăm sóc con nhỏ và những thay đổi trong cuộc sống có thể khiến bạn cảm thấy rất căng thẳng. \n\nMay mắn là có nhiều cách đơn giản mà hiệu quả để giúp bạn giảm stress sau sinh. Dưới đây là một số lời khuyên:\n\n* **Chăm sóc bản thân:** Hãy dành thời gian cho bản thân mỗi ngày, dù chỉ là 15 phút ngắn ngủi. Tắm, đọc sách, nghe nhạc, hoặc làm bất cứ điều gì bạn yêu thích để thư giãn và tái tạo năng lượng.\n* **Ăn uống lành mạnh:** Chế độ ăn uống cân bằng sẽ giúp cơ thể bạn khỏe mạnh và có sức chịu đựng tốt hơn với stress. \n* **Ngủ đủ giấc:** Thiếu ngủ sẽ làm tăng mức độ stress. Hãy cố gắng ngủ đủ 7-8 tiếng mỗi đêm, hoặc chia nhỏ giấc ngủ thành nhiều lần nếu con bạn thức suốt đêm.\n* **Tập thể dục nhẹ nhàng:** Tập thể dục có thể giúp giảm căng thẳng và nâng cao tinh thần. Bạn có thể tập yoga, đi bộ nhẹ nhàng hoặc tham gia các lớp tập thể dục dành cho bà mẹ sau sinh.\n* **Kết nối với người khác:** Chia sẻ 

c:\1THUMUCDAIHOC\HOC_KY_7\PROJECT_AI\postpartum_psychology_sup_bot\.env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


AttributeError: 'str' object has no attribute 'page_content'